# EE499 Final Project
# Premier League Predictor
Hojeong Lee and Nate Chism

In [472]:
import numpy as np
import autograd.numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from autograd import grad 
from autograd import hessian
import math
import copy
import pandas as pd
import sys
import time
from IPython.display import clear_output
readDataPath = './readData/'
sys.path.append('./')

from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import RidgeClassifier
from sklearn import svm 
from sklearn import metrics

In [473]:
season_data = pd.read_csv('./readData/df_full_premierleague.csv')
season_csv_to_array = np.array(season_data)
# print(range(0, len(house_csv_to_array) -1))
print(season_csv_to_array.shape)
print(season_csv_to_array)

(4070, 114)
[[0 'https://www.premierleague.com/match/7186' '10/11' ... 0.78 -4.0 55.6]
 [1 'https://www.premierleague.com/match/7404' '10/11' ... 0.32 17.0 60.2]
 [2 'https://www.premierleague.com/match/7255' '10/11' ... 0.38 9.0 66.7]
 ...
 [4067 'https://www.premierleague.com/match/59178' '20/21' ... 0.69 4.0
  64.1]
 [4068 'https://www.premierleague.com/match/59182' '20/21' ... 0.42 6.0
  54.2]
 [4069 'https://www.premierleague.com/match/59052' '20/21' ... 0.87 5.0
  71.1]]


In [474]:
# 2, 4, 5, 6, 8 - 31, 38 - 49, 56  
print(season_csv_to_array[5][100])

nan


In [475]:
i, j = season_csv_to_array.shape
print(i)
print(j)
match_target_scores = np.zeros((i,3))
# match_teams = np.array((i, 3))
for it_1 in range(0, i):
    # store game unique id
    match_target_scores[it_1][0] = season_csv_to_array[it_1][0]
    #store game unique id, home team, away team
    # match_teams[it_1][0] = season_csv_to_array[it_1][0]
    # match_teams[it_1][1] = season_csv_to_array[it_1][4]
    # match_teams[it_1][2] = season_csv_to_array[it_1][5]
    if season_csv_to_array[it_1][34] > 0:
        match_target_scores[it_1][1] = 1
    elif season_csv_to_array[it_1][34] < 0:
        match_target_scores[it_1][1] = -1
    else:
        match_target_scores[it_1][1] = 0

print(match_target_scores[678])

4070
114
[678.  -1.   0.]


In [476]:
season_data_cleaned = np.zeros((i, 102))

# print(season_csv_to_array[5])
season_csv_to_array[pd.isnull(season_csv_to_array)] = 0
# print(season_csv_to_array[5])
# 2, 4, 5, 6, 8 - 31, 38 - 49, 56  
for it in range(0, i):
    ct = 0
    for it2 in range(0, j):
        
        # if(np.isnan(season_csv_to_array[it][it2])):
        #     print(season_csv_to_array[it][it2])

        if it2 not in [1, 2, 3, 4, 5, 6, 7, 32, 33, 34, 35, 36, 37]:
            season_data_cleaned[it][ct] = season_csv_to_array[it][it2]
            # print(ct)
            ct = ct + 1

        elif it2 == 2:
            season_number = (int)(season_csv_to_array[it][it2][0] + season_csv_to_array[it][it2][1])
            season_data_cleaned[it][ct] = season_number
            ct = ct + 1

        
        

# print(season_data_cleaned)
print(season_data_cleaned[5])


[  5.   10.   15.    5.    9.    4.  609.   65.6   0.   19.    4.   23.
 820.    2.   32.    3.    7.    0.  313.   34.4   0.    9.    4.   21.
 527.    2.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.    0.    0.    0. ]


In [477]:
print("it: ", it, " ct: ", ct, " it2: ", it2)

it:  4069  ct:  102  it2:  113


In [478]:
print(season_data_cleaned[4000][1])

20.0


In [479]:
season_data_cleaned[123][0]

123.0

In [485]:
seasons_train = np.zeros((3419, 102))
season_18 = np.zeros((380, 102))
season_19 = np.zeros((380, 102))

Y_train = np.zeros((3419,))
Y_test1 = np.zeros((380,))
Y_test2 = np.zeros((380,))

ct1 = 0
ct2 = 0
ct3 = 0
for it in range(0, i):
    target_index = season_data_cleaned[it][0]
    if season_data_cleaned[it][1] not in [18, 19]:
        seasons_train[ct1] = season_data_cleaned[it]
        Y_train[ct1] = match_target_scores[it][1]
        ct1 = ct1 + 1
    elif season_data_cleaned[it][1] == 18:
        season_18[ct2] = season_data_cleaned[it]
        Y_test1[ct2] = match_target_scores[it][1]
        ct2 = ct2 + 1
        # print(it)
    elif season_data_cleaned[it][1] == 19:
        season_19[ct3] = season_data_cleaned[it]
        Y_test2[ct3] = match_target_scores[it][1]
        ct3 = ct3 + 1
        # print(it)
    else:
        print('THIS SHOULD NEVER HAPPEN')

In [487]:
from sklearn.preprocessing import MinMaxScaler

print(seasons_train.shape)
print(season_18.shape)
print(season_19.shape)

norm = MinMaxScaler().fit(seasons_10to18)

seasons_train_normalized = norm.transform(seasons_train)
season_18_normalized = norm.transform(season_18)
season_19_normalized = norm.transform(season_19)

(3419, 102)
(380, 102)
(380, 102)


In [ ]:
# ideas: 
    # normalize based on different probabilities
        #USE X*w0, X*w1, X*w2 for the 3 probabilities
    # train on 3 batches of 800
    # create a validation set to use for comparing models

In [488]:
X_train = seasons_train_normalized
X_test1 = season_18_normalized
X_test2 = season_19_normalized



In [489]:
#model 1: Perceptron

perc_clf = Perceptron(alpha= .0001, tol=1e-3, random_state=859)
perc_clf.fit(X_train, Y_train)
Perceptron()

print('Score: ', perc_clf.score(X_train, Y_train))

Score:  0.622696694940041


In [ ]:
# perc_clf.score(X_train, Y_train)

In [ ]:
score_hist = []
for it in range(0, 1000):
    perc_clf = Perceptron(alpha= .0001, tol=1e-3, random_state=it)
    perc_clf.fit(X_train, Y_train)
    Perceptron()
    score_hist.append(perc_clf.score(X_train, Y_train))
                      
print('Max score: ', max(score_hist))

In [ ]:
max_value_seed = max(score_hist)
max_index_seed = score_hist.index(max_value_seed)
print(max_index_seed)

In [ ]:
alpha_values = []
val = 0.0001
for i in range(0, 10000):
    alpha_values.append(val)
    val = val + 0.0001

In [ ]:
score_hist = []
for it in alpha_values:
    perc_clf = Perceptron(alpha= it, tol=1e-3, random_state=max_index_seed)
    perc_clf.fit(X_train, Y_train)
    Perceptron()
    score_hist.append(perc_clf.score(X_train, Y_train))
             
print('Max score: ', max(score_hist))
max_value_alpha = max(score_hist)
max_index_alpha = score_hist.index(max_value_alpha)
print(max_index_alpha)

In [ ]:
print('Max score: ', max(score_hist))
max_value_alpha = max(score_hist)
max_index_alpha = score_hist.index(max_value_alpha)
print(max_index_alpha)

In [491]:
perc_clf = Perceptron(alpha= .0001, tol=1e-3, random_state=859)
perc_clf.fit(X_train, Y_train)
Perceptron()

print('Perceptron')
print('Score train: ', perc_clf.score(X_train, Y_train))
print('Score test 1: ', perc_clf.score(X_test1, Y_test1))
print('Score test 2: ', perc_clf.score(X_test2, Y_test2))

Perceptron
Score train:  0.622696694940041
Score test 1:  0.6842105263157895
Score test 2:  0.6105263157894737


In [492]:
log_reg_clf = LogisticRegression(multi_class='ovr', random_state = 0, max_iter=10000, solver = 'liblinear').fit(X_train, Y_train)

print('Logistic Regression (liblinear)')
print('Score train: ', log_reg_clf.score(X_train, Y_train))
print('Score test 1: ', log_reg_clf.score(X_test1, Y_test1))
print('Score test 2: ', log_reg_clf.score(X_test2, Y_test2))

Logistic Regression (liblinear)
Score train:  0.6762211172857561
Score test 1:  0.7105263157894737
Score test 2:  0.6421052631578947


In [ ]:
# score_hist = []
# for state_num in range(0, 1000):
#     log_reg_clf = LogisticRegression(multi_class='ovr', random_state = state_num, max_iter=10000, solver = 'liblinear').fit(X_train, Y_train)

#     score_hist.append(log_reg_clf.score(X_train, Y_train))

# print('Max score: ', max(score_hist))
# max_value_seed = max(score_hist)
# max_index_seed = score_hist.index(max_value_seed)
# print('Best random_state = ', max_index_seed)

In [493]:
print('Max score: ', max(score_hist))
max_value_seed = max(score_hist)
max_index_seed = score_hist.index(max_value_seed)
print('Best random_state = ', max_index_seed)
print(len(score_hist))

Max score:  0.6762211172857561
Best random_state =  3
140


# Start Here

In [494]:
#BEST SO FAR
log_reg_clf = LogisticRegression(multi_class='ovr', random_state = 0, max_iter=10000, solver = 'newton-cg').fit(X_train, Y_train)

print('Logistic Regression (Newton)')
print('Score train: ', log_reg_clf.score(X_train, Y_train))
print('Score test 1: ', log_reg_clf.score(X_test1, Y_test1))
print('Score test 2: ', log_reg_clf.score(X_test2, Y_test2))

Logistic Regression (Newton)
Score train:  0.6735887686458029
Score test 1:  0.7131578947368421
Score test 2:  0.6421052631578947


In [495]:
print('classes: ', log_reg_clf.classes_)

print(X_test1[0].shape)
print(log_reg_clf.coef_[0].shape)
print('1: ', X_test1[41].T@log_reg_clf.coef_[0])
print('2: ', X_test1[41].T@log_reg_clf.coef_[1])
print('3: ', X_test1[41].T@log_reg_clf.coef_[2])
print('predict probabilities: ', log_reg_clf.predict_proba(X_test1)[41])
predictions1 = log_reg_clf.predict(X_test1)
predictions2 = log_reg_clf.predict(X_test2)


classes:  [-1.  0.  1.]
(102,)
(102,)
1:  2.697605849575331
2:  -4.9945212883351235
3:  4.158158258526025
predict probabilities:  [0.17238273 0.24843513 0.57918213]


In [496]:
print('Accuracy Score for season 1: ', metrics.accuracy_score(predictions1, Y_test1))
print('Balanced Accuracy Score for season 1: ', metrics.balanced_accuracy_score(predictions1, Y_test1))
print('Accuracy Score for season 2: ', metrics.accuracy_score(predictions2, Y_test2))
print('Balanced Accuracy Score for season 2: ', metrics.balanced_accuracy_score(predictions2, Y_test2))

Accuracy Score for season 1:  0.7131578947368421
Balanced Accuracy Score for season 1:  0.5922598991961139
Accuracy Score for season 2:  0.6421052631578947
Balanced Accuracy Score for season 2:  0.6110492577597841


In [497]:
prob1 = log_reg_clf.predict_proba(X_test1)
temp0 = prob1[0][0]
temp1 = prob1[0][1]
temp2 = prob1[0][2]
print('==========')

rand_num = np.random.random()

print(temp0)
print(temp1)
print(temp2)

thresh0 = temp0
thresh1 = temp0+temp1
thresh2 = temp0 + temp1 + temp2
print('==========')
print(thresh0, ',', thresh1, ',', thresh2)
print('==========')
if rand_num < thresh0:
    print(rand_num)
    print('-1')
elif thresh0 < rand_num < thresh1:
    print(rand_num)
    print('0')
else:
    print(rand_num)
    print('+1')



0.09753230111739626
0.0960201258919822
0.8064475729906216
0.09753230111739626 , 0.19355242700937847 , 1.0
0.5018519079614863
+1


In [ ]:
print(np.random.random())

In [498]:
prob1 = log_reg_clf.predict_proba(X_test1)
prob2 = log_reg_clf.predict_proba(X_test2)

max_pred1 = np.zeros(len(X_test1))
random_pred1 = np.zeros(len(X_test1))
max_pred2 = np.zeros(len(X_test2))
random_pred2 = np.zeros(len(X_test2))

for it in range(0, len(X_test1)):
    max_pred1[it] = np.max(predictions1[it])
    temp0 = prob1[it][0]
    temp1 = prob1[it][1]
    temp2 = prob1[it][2]
    pick = np.random.random()

    thresh0 = temp0
    thresh1 = temp0+temp1
    thresh2 = temp0 + temp1 + temp2
    # print('==========')
    # print(thresh0, ',', thresh1, ',', thresh2)
    # print('==========')
    if pick < thresh0:
        # print(rand_num)
        # print('-1')
        random_pred1[it] = -1
    elif thresh0 < pick < thresh1:
        # print(rand_num)
        # print('0')
        random_pred1[it] = 0
    else:
        # print(rand_num)
        # print('+1')
        random_pred1[it] = 1

for it in range(0, len(X_test2)):
    max_pred2[it] = np.max(predictions2[it])
    temp0 = prob2[it][0]
    temp1 = prob2[it][1]
    temp2 = prob2[it][2]
    pick = np.random.random()

    thresh0 = temp0
    thresh1 = temp0+temp1
    thresh2 = temp0 + temp1 + temp2
    # print('==========')
    # print(thresh0, ',', thresh1, ',', thresh2)
    # print('==========')
    if pick < thresh0:
        # print(rand_num)
        # print('-1')
        random_pred2[it] = -1
    elif thresh0 < pick < thresh1:
        # print(rand_num)
        # print('0')
        random_pred2[it] = 0
    else:
        # print(rand_num)
        # print('+1')
        random_pred2[it] = 1

model_pred1 = log_reg_clf.predict(X_test1)
model_pred2 = log_reg_clf.predict(X_test2)

In [499]:
print('Model Accuracy Score for season 1:  ', metrics.accuracy_score(model_pred1, Y_test1))
print('Max Accuracy Score for season 1:    ', metrics.accuracy_score(max_pred1, Y_test1))
print('Random Accuracy Score for season 1: ', metrics.accuracy_score(random_pred1, Y_test1))
print('\n')
print('Model Accuracy Score for season 2:  ', metrics.accuracy_score(model_pred2, Y_test2))
print('Max Accuracy Score for season 2:    ', metrics.accuracy_score(max_pred2, Y_test2))
print('Random Accuracy Score for season 2: ', metrics.accuracy_score(random_pred2, Y_test2))


Model Accuracy Score for season 1:   0.7131578947368421
Max Accuracy Score for season 1:     0.7131578947368421
Random Accuracy Score for season 1:  0.5421052631578948


Model Accuracy Score for season 2:   0.6421052631578947
Max Accuracy Score for season 2:     0.6421052631578947
Random Accuracy Score for season 2:  0.5026315789473684


In [500]:
# even size classes
count = 0
count1 = 0
count2 = 0
big_ct = 0
X_train_trunc = np.zeros((2400, 102))
Y_train_trunc = np.zeros(2400)


for it in range(len(Y_train)):
    index = np.random.randint(0, 3419)
    if Y_train[it] == 0:
        if count < 800:
            X_train_trunc[big_ct] = X_train[index]
            Y_train_trunc[big_ct] = Y_train[index]
            big_ct = big_ct + 1
        count = count + 1
    if Y_train[it] == 1:
        if count1 < 800:
            X_train_trunc[big_ct] = X_train[index]
            Y_train_trunc[big_ct] = Y_train[index]
            big_ct = big_ct + 1
        count1 = count1 + 1
    if Y_train[it] == -1:
        if count2 < 800:
            X_train_trunc[big_ct] = X_train[index]
            Y_train_trunc[big_ct] = Y_train[index]
            big_ct = big_ct + 1
        count2 = count2 + 1

print(count)
print(count1)
print(count2)
print(count + count1 + count2)
print(len(Y_train))

count = 0
count1 = 0
count2 = 0

for it in range(len(Y_train_trunc)):
    if Y_train_trunc[it] == 0:
        # if count < 800:
        #     X_train_trunc[big_ct] = X_train[it]
        #     Y_train_trunc[big_ct] = Y_train[it]
        #     big_ct = big_ct + 1
        count = count + 1
    if Y_train_trunc[it] == 1:
        # if count1 < 800:
        #     X_train_trunc[big_ct] = X_train[it]
        #     Y_train_trunc[big_ct] = Y_train[it]
        #     big_ct = big_ct + 1
        count1 = count1 + 1
    if Y_train_trunc[it] == -1:
        # if count2 < 800:
        #     X_train_trunc[big_ct] = X_train[it]
        #     Y_train_trunc[big_ct] = Y_train[it]
        #     big_ct = big_ct + 1
        count2 = count2 + 1

print('==========')
print(count)
print(count1)
print(count2)
print(count + count1 + count2)
print(len(Y_train_trunc))

946
1482
991
3419
3419
655
1093
652
2400
2400


In [501]:
# even size classses
log_reg_clf = LogisticRegression(multi_class='ovr', random_state = 0, max_iter=10000, solver = 'newton-cg').fit(X_train_trunc, Y_train_trunc)

print('Logistic Regression (Newton)')
print('Score train: ', log_reg_clf.score(X_train_trunc, Y_train_trunc))
print('Score test 1: ', log_reg_clf.score(X_test1, Y_test1))
print('Score test 2: ', log_reg_clf.score(X_test2, Y_test2))

Logistic Regression (Newton)
Score train:  0.7041666666666667
Score test 1:  0.718421052631579
Score test 2:  0.6289473684210526


the rankings

In [502]:
#BEST SO FAR
log_reg_clf = LogisticRegression(multi_class='ovr', random_state = 0, max_iter=10000, solver = 'newton-cg').fit(X_train, Y_train)

print('Logistic Regression (Newton)')
print('Score train: ', log_reg_clf.score(X_train, Y_train))
print('Score test 1: ', log_reg_clf.score(X_test1, Y_test1))
print('Score test 2: ', log_reg_clf.score(X_test2, Y_test2))

Logistic Regression (Newton)
Score train:  0.6735887686458029
Score test 1:  0.7131578947368421
Score test 2:  0.6421052631578947


In [512]:
y_pred1 = log_reg_clf.predict(X_test1)
y_pred2 = log_reg_clf.predict(X_test2)

# 2018-2019

In [513]:
standing18 = {}
indexofteams = []
for i in range(len(season_csv_to_array)):
    season_number = (int)(season_csv_to_array[i][2][0] + season_csv_to_array[i][2][1])
    # print(season_number)
    if season_number == 18:
        indexofteams.append(i)
        if (season_csv_to_array[i][4]) in standing18:
            standing18[season_csv_to_array[i][4]] = 0
            # print(season_csv_to_array[i][4])
        else:
            standing18[season_csv_to_array[i][4]] = 0

In [524]:
it = 1
for key in standing18:
    print(it)
    print(key)
    it = it + 1

1
Manchester United
2
Leicester City
3
Wolverhampton Wanderers
4
Cardiff City
5
West Ham United
6
Liverpool
7
Everton
8
Southampton
9
Brighton and Hove Albion
10
Arsenal
11
AFC Bournemouth
12
Manchester City
13
Watford
14
Crystal Palace
15
Chelsea
16
Burnley
17
Tottenham Hotspur
18
Huddersfield Town
19
Newcastle United
20
Fulham


In [515]:
i= 2664
for ind in range(len(y_pred1)):
    if y_pred1[ind] == 1:
        standing18[season_csv_to_array[i][4]] += 3
    elif y_pred1[ind] == 0:
        standing18[season_csv_to_array[i][4]] += 1
        standing18[season_csv_to_array[i][5]] += 1
    else:
        standing18[season_csv_to_array[i][5]] += 3
    i=i+1

KeyError: 'Stoke City'

In [ ]:
sorted_dict19 = dict(sorted(standing18.items(), key=lambda x: x[1], reverse=True))
print(sorted_dict19)
ct = 0
print('2019-2020 Premier League Predicted Standings')
for it in sorted_dict19:
    print(ct, '\t', it, ':', sorted_dict19[it])
    ct = ct+1

In [ ]:
actual_values_19 = {
            'Liverpool' : (1, 99),
            'Manchester City' : (2, 81),
            'Manchester United' : (3, 66),
            'Chelsea' : (4, 66),
            'Leicester City' : (5, 62),
            'Tottenham Hotspur' : (6, 59),
            'Wolverhampton Wanderers' : (7, 59),
            'Arsenal' : (8, 56),
            'Sheffield United' : (9, 54),
            'Burnley' : (10, 54),
            'Southampton' : (11, 52),
            'Everton' : (12, 49),
            'Newcastle United' : (13, 44),
            'Crystal Palace' : (14, 43),
            'Brighton and Hove Albion' : (15, 41),
            'West Ham United' : (16, 39),
            'Aston Villa' : (17, 35),
            'AFC Bournemouth' : (18, 34),
            'Watford' : (19, 34),
            'Norwich City' : (20, 21),
}

In [ ]:
max_len_key = max(len(k) for k in sorted_dict19.keys())  # get the maximum length of keys
max_len_value = max(len(str(v)) for v in sorted_dict19.values())  # get the maximum length of values
place = 1

print('2019-2020 Premier League Season')
print('# \t Team \t\t\t\t Points (pred)\tPoints (real)\tPoints Diff\tPlace Diff')
print('__________________________________________________________________________________________________')
for key, value in sorted_dict19.items():
    if place < 10:
        print(place, ' :\t', "{:{key_width}}  \t {:{value_width}} \t\t {:{value_width}}\t\t {:{value_width}}\t\t {:{value_width}}".format(key, value, actual_values_19[key][1], value - actual_values_19[key][1], actual_values_19[key][0] - place, key_width=max_len_key, value_width=max_len_value))
    else:
        print(place, ':\t', "{:{key_width}}  \t {:{value_width}} \t\t {:{value_width}}\t\t {:{value_width}}\t\t {:{value_width}}".format(key, value, actual_values_19[key][1], value - actual_values_19[key][1], actual_values_19[key][0] - place, key_width=max_len_key, value_width=max_len_value))
    place = place + 1

In [ ]:
champions_leage_pred = []
champions_league_real = ['Liverpool', 'Manchester City', 'Manchester United', 'Chelsea']
relegation_pred = []
relegation_real = ['AFC Bournemouth', 'Watford', 'Norwich City']

place = 1
for key, value in sorted_dict19.items():
    if place < 5:
        champions_leage_pred.append(key)
    if place > 17:
        relegation_pred.append(key)
    place = place + 1
print('2019-2020 Premier League Season')
print('Champion League Qualification')
print('Predicted Teams \t\t Actual Teams')
print('_________________________________________________')
for it in range(0, len(champions_leage_pred)):
    print("{:{key_width}}\t{:{key_width}}".format(champions_leage_pred[it], champions_league_real[it], key_width=max_len_key))
print('\n')
print('Relegation Battle')
print('Predicted Teams \t\t Actual Teams')
print('_________________________________________________')
for it in range(len(relegation_pred)):
    print("{:{key_width}}\t{:{key_width}}".format(relegation_pred[it], relegation_real[it], key_width=max_len_key))

# 2019-2020

In [ ]:
standing19 = {}
indexofteams = []
for i in range(len(season_csv_to_array)):
    season_number = (int)(season_csv_to_array[i][2][0] + season_csv_to_array[i][2][1])
    if season_number == 19:
        indexofteams.append(i)
        if (season_csv_to_array[i][4]) in standing19:
            standing19[season_csv_to_array[i][4]] = 0
            # print(season_csv_to_array[i][4])
        else:
            standing19[season_csv_to_array[i][4]] = 0

In [ ]:
i= 3799
for ind in range(len(y_pred2)):
    if y_pred2[ind] == 1:
        standing19[season_csv_to_array[i][4]] += 3
    elif y_pred2[ind] == 0:
        standing19[season_csv_to_array[i][4]] += 1
        standing19[season_csv_to_array[i][5]] += 1
    else:
        standing19[season_csv_to_array[i][5]] += 3
    i=i+1

In [ ]:
sorted_dict20 = dict(sorted(standing19.items(), key=lambda x: x[1], reverse=True))
print(sorted_dict20)
ct = 0
print('2019-2020 Premier League Predicted Standings')
for it in sorted_dict20:
    print(ct, '\t', it, ':', sorted_dict20[it])
    ct = ct+1

In [ ]:
actual_values_19 = {
            'Liverpool' : (1, 99),
            'Manchester City' : (2, 81),
            'Manchester United' : (3, 66),
            'Chelsea' : (4, 66),
            'Leicester City' : (5, 62),
            'Tottenham Hotspur' : (6, 59),
            'Wolverhampton Wanderers' : (7, 59),
            'Arsenal' : (8, 56),
            'Sheffield United' : (9, 54),
            'Burnley' : (10, 54),
            'Southampton' : (11, 52),
            'Everton' : (12, 49),
            'Newcastle United' : (13, 44),
            'Crystal Palace' : (14, 43),
            'Brighton and Hove Albion' : (15, 41),
            'West Ham United' : (16, 39),
            'Aston Villa' : (17, 35),
            'AFC Bournemouth' : (18, 34),
            'Watford' : (19, 34),
            'Norwich City' : (20, 21),
}

In [ ]:
max_len_key = max(len(k) for k in sorted_dict19.keys())  # get the maximum length of keys
max_len_value = max(len(str(v)) for v in sorted_dict19.values())  # get the maximum length of values
place = 1

print('2019-2020 Premier League Season')
print('# \t Team \t\t\t\t Points (pred)\tPoints (real)\tPoints Diff\tPlace Diff')
print('__________________________________________________________________________________________________')
for key, value in sorted_dict19.items():
    if place < 10:
        print(place, ' :\t', "{:{key_width}}  \t {:{value_width}} \t\t {:{value_width}}\t\t {:{value_width}}\t\t {:{value_width}}".format(key, value, actual_values_19[key][1], value - actual_values_19[key][1], actual_values_19[key][0] - place, key_width=max_len_key, value_width=max_len_value))
    else:
        print(place, ':\t', "{:{key_width}}  \t {:{value_width}} \t\t {:{value_width}}\t\t {:{value_width}}\t\t {:{value_width}}".format(key, value, actual_values_19[key][1], value - actual_values_19[key][1], actual_values_19[key][0] - place, key_width=max_len_key, value_width=max_len_value))
    place = place + 1

# End Here

In [ ]:
log_reg_clf = LogisticRegressionCV(multi_class='ovr', random_state = 13, max_iter=1000, solver = 'liblinear').fit(X_train, Y_train)

print('Logistic RegressionCV (newton-cholesky)')
print('Score train: ', log_reg_clf.score(X_train, Y_train))
print('Score test 1: ', log_reg_clf.score(X_test1, Y_test1))
print('Score test 2: ', log_reg_clf.score(X_test2, Y_test2))

In [ ]:
log_reg_clf = RidgeClassifier(alpha=0.001, random_state = 13, max_iter=1000, solver = 'lbfgs', positive=True).fit(X_train, Y_train)

print('Ridge Classifier (auto)')
print('Score train: ', log_reg_clf.score(X_train, Y_train))
print('Score test 1: ', log_reg_clf.score(X_test1, Y_test1))
print('Score test 2: ', log_reg_clf.score(X_test2, Y_test2))

In [ ]:
score_hist = []
# nonlinear SVM
for seed in range(1, 100):
    svm_clf = svm.SVC(gamma='scale', degree=3, random_state=seed)
    svm_clf.fit(X_train, Y_train)

    score_hist.append(svm_clf.score(X_train, Y_train))



In [ ]:
svm_clf = svm.SVC(gamma='scale', degree=2, random_state=0, kernel='poly')
svm_clf.fit(X_train, Y_train)

print('SVM SVC')
# print('degree = ', deg)
print('Score train: ', svm_clf.score(X_train, Y_train))
print('Score test 1: ', svm_clf.score(X_test1, Y_test1))
print('Score test 2: ', svm_clf.score(X_test2, Y_test2))

In [ ]:
svm_clf = svm.LinearSVC(penalty= 'l2', loss= 'hinge',random_state=seed)
svm_clf.fit(X_train, Y_train)

score_hist.append(svm_clf.score(X_train, Y_train))

print('SVM LinearSVC')
print('Score train: ', svm_clf.score(X_train, Y_train))
print('Score test 1: ', svm_clf.score(X_test1, Y_test1))
print('Score test 2: ', svm_clf.score(X_test2, Y_test2))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

weights = {-1:1.0, 0:1.0, -1:1.0}

train_results_hist_depth = []
test_results_hist_depth1 = []
test_results_hist_depth2 = []

for deep in range(1,20):
    rfc = RandomForestClassifier(random_state = 42, max_depth=18, criterion='gini', class_weight=weights, n_estimators=deep)
    rfc.fit(X_train, Y_train)
    y_pred_train = rfc.predict(X_train) 
    y_pred_test1 = rfc.predict(X_test1)  
    y_pred_test2 = rfc.predict(X_test2)

    train_results_hist_depth.append(metrics.balanced_accuracy_score(Y_train, y_pred_train))
    test_results_hist_depth1.append(metrics.balanced_accuracy_score(Y_test1, y_pred_test1))
    test_results_hist_depth2.append(metrics.balanced_accuracy_score(Y_test2, y_pred_test2)) 



In [ ]:
plt.plot(range(1,20), train_results_hist_depth, 'xb-', label='train')
plt.plot(range(1,20), test_results_hist_depth1, 'sy-', label='test1')
plt.plot(range(1,20), test_results_hist_depth2, 'sy-', label='test2')
plt.title('Accuracy vs Min Samples Leaf')
plt.xlabel('Min Samples Split')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#gradient boosting!
from sklearn.ensemble import GradientBoostingClassifier

gd_boost_clf = GradientBoostingClassifier(n_estimators=7, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, Y_train)

print('GradientBoostingClassifier')
print('Score train: ', gd_boost_clf.score(X_train, Y_train))
print('Score test 1: ', gd_boost_clf.score(X_test1, Y_test1))
print('Score test 2: ', gd_boost_clf.score(X_test2, Y_test2))